In [168]:
import pandas as pd

# Reading the CSV file 'coinbase.csv' into a pandas DataFrame
df = pd.read_csv('coinbase.csv')

# Converting Unix timestamps in the 'timestamp' column to a human-readable datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

# Sorting the DataFrame first by 'product_id' and then by 'timestamp' in ascending order
df = df.sort_values(by=['product_id', 'timestamp'], ascending=[False, True])

# Renaming the column 'product_id' to 'coin_id' for consistency
df = df.rename(columns={'product_id': 'coin_id'})

# Count the occurrences of each coin_id
coin_id_counts = df['coin_id'].value_counts()

# Find coin_ids that appear more than 730 times
coin_ids_over_730 = coin_id_counts[coin_id_counts > 1055].index

# Filter to keep only rows with these coin_ids
df = df[df['coin_id'].isin(coin_ids_over_730)]

# Remove USD from each coin name
df['coin_id'] = df['coin_id'].str.replace('-USD', '', regex=False)

# Make strings lower case 
df['coin_id'] = df['coin_id'].str.lower()

#check null values 
df.isnull().sum()

coin_id      0
timestamp    0
low          0
high         0
open         0
close        0
volume       0
dtype: int64

In [169]:
# Read the CSV file containing the Federal Reserve interest rate data
fed = pd.read_csv('Alpha_interest_yield_daily.csv')

# Rename columns in the 'fed' DataFrame for clarity and consistency
fed = fed.rename(columns={
    'date': 'timestamp', 
    'y_3month': '3m_yield', 
    'y_2year': '2yr_yield', 
    'y_5year': '5yr_yield', 
    'y_7year': '7yr_yield', 
    'y_10year': '10yr_yield', 
    'y_30year': '30yr_yield'
})

# Convert the 'timestamp' column to datetime format for proper time series merging
fed.timestamp = pd.to_datetime(fed.timestamp)

# sort by dates
fed = fed.sort_values(by=['timestamp'], ascending=[True])


columns_to_convert = fed.columns.drop('timestamp')

# Convert columns to numeric
fed[columns_to_convert] = fed[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [170]:
# Filling in missing days to interpolate

# set timestamp as index

fed = fed.set_index('timestamp')

# Create a complete date range from 01-01-2021 to 11-23-2023
date_range = pd.date_range(start='2021-01-01', end='2023-11-23', freq='D')

# Reindex the DataFrame with the date range
fed = fed.reindex(date_range)

fed = fed.reset_index().rename(columns={'index': 'timestamp'})

fed.isnull().sum()

timestamp               0
3m_yield              332
2yr_yield             332
5yr_yield             332
7yr_yield             332
10yr_yield            332
30yr_yield            332
federal_funds_rate    302
dtype: int64

In [171]:
# Select columns interpolation
numeric_cols = fed.select_dtypes(include=['number']).columns

# Apply linear interpolation 
fed[numeric_cols] = fed[numeric_cols].interpolate(method='linear')

# Apply backward fill on rest 
fed.bfill(inplace=True)

fed.isnull().sum()

timestamp             0
3m_yield              0
2yr_yield             0
5yr_yield             0
7yr_yield             0
10yr_yield            0
30yr_yield            0
federal_funds_rate    0
dtype: int64

In [172]:
df.coin_id.unique()

array(['zrx', 'zec', 'yfi', 'wbtc', 'uni', 'uma', 'snx', 'link', 'knc',
       'fil', 'eth', 'etc', 'dai', 'btc', 'atom', 'algo', 'aave'],
      dtype=object)

In [173]:
# Read MarketCap Data 
df_2 = pd.read_csv('CoinGecko.csv')

# Converting the 'timestamp' column to datetime 
df_2.timestamp = pd.to_datetime(df_2.timestamp)

rename_dict = {'bitcoin':'btc', 'ethereum': 'eth', '0x': 'zrx', 'zcash':'zec', 
 'yearn-finance': 'yfi', 'uniswap': 'uni', 'havven': 'snx', 
 'chainlink': 'link', 'kyber-network-crystal': 'knc', 
 'wrapped-bitcoin': 'wbtc', 'filecoin': 'fil', 
 'ethereum-classic': 'etc','cosmos': 'atom', 'algorand': 'algo'}

# Map the 'coin_id' column to the new values, keeping original values where no mapping is found
df_2['coin_id'] = df_2['coin_id'].map(rename_dict).fillna(df_2['coin_id'])

df_2

,coin_id,timestamp,Market Cap
0,btc,2021-01-01,5.455933e+11
1,btc,2021-01-02,5.978877e+11
2,btc,2021-01-03,6.136169e+11
3,btc,2021-01-04,5.857263e+11
4,btc,2021-01-05,6.336517e+11
...,...,...,...
10430,knc,2023-11-19,1.205137e+08
10431,knc,2023-11-20,1.169595e+08
10432,knc,2023-11-21,1.080592e+08
10433,knc,2023-11-22,1.107972e+08


In [174]:
df_2.isnull().sum()

coin_id       0
timestamp     0
Market Cap    0
dtype: int64

In [175]:
# Read More MarketCap Data 
df_3 = pd.read_csv('CoinGecko2.csv')

# Converting the 'timestamp' column to datetime
df_3.timestamp = pd.to_datetime(df_3.timestamp)

rename_dict = {'bitcoin':'btc', 'ethereum': 'eth', '0x': 'zrx', 'zcash':'zec', 
 'yearn-finance': 'yfi', 'uniswap': 'uni', 'havven': 'snx', 
 'chainlink': 'link', 'kyber-network-crystal': 'knc', 
 'wrapped-bitcoin': 'wbtc', 'filecoin': 'fil', 
 'ethereum-classic': 'etc','cosmos': 'atom', 'algorand': 'algo'}

# Map the 'coin_id' column to the new values, keeping original values where no mapping is found
df_3['coin_id'] = df_3['coin_id'].map(rename_dict).fillna(df_3['coin_id'])

df_3

,coin_id,timestamp,Market Cap
0,wbtc,2021-01-01,3.316038e+09
1,wbtc,2021-01-02,3.458108e+09
2,wbtc,2021-01-03,3.622438e+09
3,wbtc,2021-01-04,3.411655e+09
4,wbtc,2021-01-05,3.713152e+09
...,...,...,...
9373,btc,2023-11-19,7.325218e+11
9374,btc,2023-11-20,7.326735e+11
9375,btc,2023-11-21,7.027059e+11
9376,btc,2023-11-22,7.322597e+11


In [176]:
df_3.isnull().sum()

coin_id       0
timestamp     0
Market Cap    0
dtype: int64

In [184]:
# Merge Marketcap Data
merged_cap = pd.merge(df_2, df_3, on=['timestamp', 'coin_id', 'Market Cap'], how='outer')

btc     1057
eth     1057
algo    1057
atom    1057
etc     1057
fil     1057
dai     1057
wbtc    1057
link    1057
snx     1057
uma     1057
uni     1057
yfi     1057
zec     1057
zrx     1057
aave    1057
knc      922
Name: coin_id, dtype: int64

In [189]:
# Merge crypto Price Data
merged_crypto = pd.merge(merged_cap, df, on=['timestamp', 'coin_id'], how='outer')
null = merged_crypto[merged_crypto['Market Cap'].isnull()]
null

,coin_id,timestamp,Market Cap,low,high,open,close,volume
17868,knc,2021-01-01,NaN,0.7794,0.8401,0.7994,0.8012,1793700.6
17869,knc,2021-01-02,NaN,0.7813,0.8140,0.8013,0.7828,1861391.1
17870,knc,2021-01-03,NaN,0.7648,0.8442,0.7841,0.8318,3226370.4
17871,knc,2021-01-04,NaN,0.7674,0.9384,0.8319,0.8230,5817576.0
17872,knc,2021-01-05,NaN,0.7999,0.9260,0.8230,0.9135,4715277.5
...,...,...,...,...,...,...,...,...
17998,knc,2021-05-13,NaN,2.7577,3.1841,2.9230,2.9859,4422671.3
17999,knc,2021-05-14,NaN,2.9700,3.2565,2.9830,3.1580,2195160.3
18000,knc,2021-05-15,NaN,2.8068,3.1993,3.1550,2.8284,2485139.5
18001,knc,2021-05-16,NaN,2.5601,3.0354,2.8274,2.7120,2369431.5


In [207]:
# Sorting the DataFrame first by 'coin_id' and then by 'timestamp' in ascending order
merged_crypto = merged_crypto.sort_values(by=['coin_id', 'timestamp'], ascending=[True, True])

# Force 0's to NaN
merged_crypto.loc[merged_crypto['Market Cap'] == 0, 'Market Cap'] = pd.NA

# Apply linear interpolation to the Market Cap
merged_crypto['Market Cap'] = merged_crypto['Market Cap'].interpolate(method='linear')

In [213]:
# Merge the cryptocurrency data and interest rate and US treasury yield data 
merged_df = pd.merge(merged_crypto, fed, on='timestamp', how='outer')

In [214]:
# Sorting the DataFrame first by 'coin_id' and then by 'timestamp' in ascending order
merged_df = merged_df.sort_values(by=['coin_id', 'timestamp'], ascending=[True, True])
merged_df

,coin_id,timestamp,Market Cap,low,high,open,close,volume,3m_yield,2yr_yield,5yr_yield,7yr_yield,10yr_yield,30yr_yield,federal_funds_rate
0,aave,2021-01-01,1.079073e+09,84.920000,91.127000,88.614000,91.119000,5.576249e+04,0.090000,0.110000,0.360000,0.640000,0.930000,1.660000,0.09
17,aave,2021-01-02,1.031438e+09,82.680000,92.443000,91.074000,85.825000,5.352018e+04,0.090000,0.110000,0.360000,0.640000,0.930000,1.660000,0.09
34,aave,2021-01-03,1.144026e+09,81.596000,95.130000,85.867000,93.860000,8.504981e+04,0.090000,0.110000,0.360000,0.640000,0.930000,1.660000,0.09
51,aave,2021-01-04,1.307178e+09,84.488000,116.131000,94.044000,114.359000,1.931996e+05,0.090000,0.110000,0.360000,0.640000,0.930000,1.660000,0.09
68,aave,2021-01-05,1.444938e+09,98.000000,124.220000,114.343000,119.923000,1.872953e+05,0.090000,0.130000,0.380000,0.660000,0.960000,1.700000,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17934,zrx,2023-11-19,3.731381e+07,0.384040,0.438907,0.399452,0.400420,7.686476e+06,5.526667,4.886667,4.443333,4.463333,4.426667,4.576667,5.33
17951,zrx,2023-11-20,3.655758e+07,0.383962,0.404714,0.400472,0.396725,3.900178e+06,5.540000,4.890000,4.440000,4.460000,4.420000,4.570000,5.33
17968,zrx,2023-11-21,3.481796e+07,0.368265,0.409390,0.396336,0.370301,5.110680e+06,5.530000,4.860000,4.410000,4.440000,4.410000,4.570000,5.33
17985,zrx,2023-11-22,3.749816e+07,0.366536,0.411433,0.369717,0.404362,3.018978e+06,5.540000,4.890000,4.440000,4.460000,4.420000,4.550000,5.33


In [215]:
# Save the merged DataFrame to a CSV file named 'final.csv' without including the index
merged_df.to_csv('final.csv', index=False)